In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

# Image Capture

In [ ]:
fps = 10
frame_width = 640
frame_height = 480

In [ ]:
def run_image_capture(process_fun):
    global frame_width
    global frame_height
    try:
        cap = cv2.VideoCapture(0)
        cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
        cap.set(cv2.CAP_PROP_FRAME_HEIGHT, frame_height)
        while cv2.waitKey(int(1 / fps * 1000)) != 27:
            ret, frame = cap.read()
            if ret:
                process_fun(frame)
    finally:
        cap.release()
        del cap
        cv2.destroyAllWindows()

Run video capture at 10 FPS and configure camera to run with 640px frame width.

In [ ]:
run_image_capture(lambda frame: cv2.imshow("Webcam", frame))

Determine pixel data-type.

In [ ]:
try:
    cap = cv2.VideoCapture(0)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, frame_width)
    _, frame = cap.read()
finally:
    cap.release()
    del cap
    cv2.destroyAllWindows()

print(f"Type: {type(frame[0, 0]).__name__}")
print(f"Shape: {frame[0, 0].shape}")

Draw Region of Interrest (ROI) around face.

In [ ]:
roi = frame.copy()
roi[50, 150:-150] = np.full_like(roi[50, 150:-150], [0, 0, 255])
roi[-100, 150:-150] = np.full_like(roi[-100, 150:-150], [0, 0, 255])
roi[50:-100, 150] = np.full_like(roi[50:-100, 150], [0, 0, 255])
roi[50:-100, -150] = np.full_like(roi[50:-100, -150], [0, 0, 255])
cv2.imshow("Region of Interest", roi)
cv2.waitKey(0)
cv2.destroyAllWindows()

Display color channels separately.

In [ ]:
for i, title in enumerate(["Blue", "Green", "Red"]):
    cv2.imshow(title, frame[:, :, i])

cv2.waitKey(0)
cv2.destroyAllWindows()

Run grayscale webcam capture.

In [ ]:
gray_histogram = []


def convert_to_grayscale(frame):
    global gray_histogram
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    gray_histogram.append(np.mean(gray[int(gray.shape[0] / 2), :]))
    cv2.imshow("Grayscale", gray)


run_image_capture(convert_to_grayscale)

ax = plt.subplot()
ax.plot(np.arange(len(gray_histogram)) / fps, gray_histogram)
ax.set_title("Grayscale Histogram")
ax.set_xlabel("Time [s]")
ax.set_ylabel("Gray Intensity")
ax.set_ylim([0, 255])
ax.grid(True)

# Image Processing

Increase brightness of image via `cv2.add()` and calculate image delta between frames via `cv2.absdiff()`.

In [ ]:
last_gray = None


def apply_add_and_delta(frame):
    global last_gray
    cv2.imshow("add()", cv2.add(frame, 100))
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    if last_gray is not None:
        cv2.imshow("absdiff()", cv2.absdiff(gray, last_gray))
    last_gray = gray


run_image_capture(apply_add_and_delta)

Apply blur and Sobel filter to image.

In [ ]:
image = cv2.imread("data/solarzelle.bmp", 0)

fig = plt.figure(figsize=(20, 5))

blur = cv2.blur(image, (5, 5))
ax = fig.add_subplot(1, 3, 1)
ax.imshow(blur, cmap="gray")
ax.set_title("Blur")

sobel = cv2.Sobel(image, cv2.CV_8U, 1, 0, ksize=3)
ax = fig.add_subplot(1, 3, 2)
ax.imshow(sobel, cmap="gray")
ax.set_title("Sobel (horizontal)")

sobel = cv2.Sobel(image, cv2.CV_8U, 0, 1, ksize=3)
ax = fig.add_subplot(1, 3, 3)
ax.imshow(sobel, cmap="gray")
ax.set_title("Sobel (vertical)")

Apply all-ones 3x3 kernel to image.

In [ ]:
fig = plt.figure(figsize=(20, 5))

image = np.zeros((10, 10))
image[2:-2, 2:-2] = np.full_like(image[2:-2, 2:-2], 9)
ax = fig.add_subplot(1, 4, 1)
ax.imshow(image, cmap="gray")
ax.set_title("Image")

kernel = np.ones([3, 3])
conv = cv2.filter2D(src=image, ddepth=-1, kernel=kernel) / 9
ax = fig.add_subplot(1, 4, 2)
ax.imshow(conv, cmap="gray")
ax.set_title("All-ones Kernel")

blur = cv2.blur(image, (3, 3))
ax = fig.add_subplot(1, 4, 3)
ax.imshow(blur, cmap="gray")
ax.set_title("Blur")

gaus = cv2.GaussianBlur(image, (3, 3), 0)
ax = fig.add_subplot(1, 4, 4)
ax.imshow(gaus, cmap="gray")
ax.set_title("Gaussian-Blur")